In [4]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 848 kB/s eta 0:00:01


In [19]:
from keras.layers import Dense, Input, LSTM, Embedding,Dropout,Bidirectional,Flatten
from keras.layers import*
from keras.engine.topology import Layer
from keras import backend as K
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras
import h5py
from keras.optimizers import RMSprop



def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "lstm100_bi_ad_age_pre200_true_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")


def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    main_input = Input(shape=(200,), name='main_input', dtype='int32')
    
    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/embeddings_matrix_ad_size_32_w200_count_0.h5','r') as f:
        embeddings_matrix = np.array(f.get('embeddings_matrix'))
    embeddings = Embedding(input_dim = len(embeddings_matrix), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input)
    
    lstm_out = Bidirectional(LSTM(32,activation='softsign',return_sequences=False))(embeddings)
#     lstm_out = Bidirectional(LSTM(10,activation='softsign',return_sequences=False))(lstm_out)
    
    # 堆叠全连接
#     den_x = Dense(32, activation='relu')(lstm_out)
#     # 辅助输入数据
#     auxiliary_input = Input(shape=(48,), name='aux_input')
#     # 将辅助输入与lstm输出连接起来
#     den_x = keras.layers.concatenate([den_x, auxiliary_input])
    # 辅助损失函数
#     auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)
    # 主输出
    
    main_output = Dense(10, activation='softmax', name='main_output')(lstm_out)
    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[main_input],#,auxiliary_input],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    
    opt = RMSprop(lr=0.01,  clipnorm=1.0)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])

    print('lstm model geted...\n')
    return model


def data_load():
    print('loading data ... \n')

    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/word_train_ad_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

def load_data_sta():
    data_sta = pd.read_csv('../../../train_feat_statisc.csv').sort_values(by = 'user_id')
    
    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])
    data_sta = label.merge(data_sta, on='user_id', how='left').sort_values(by='user_id')
    data_sta.fillna(0)
    data_sta = data_sta.drop(['Unnamed: 0','user_id','gender','age'], axis=1)

    train_x, test_x= train_test_split(data_sta, test_size=0.2, random_state=2020)
    return train_x, test_x

In [20]:
model = buid_model()

lstm model start...

lstm model geted...



In [21]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 32)           32319296  
_________________________________________________________________
bidirectional_8 (Bidirection (None, 64)                16640     
_________________________________________________________________
main_output (Dense)          (None, 10)                650       
Total params: 32,336,586
Trainable params: 32,336,586
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

# train_x_sta, test_x_sta = load_data_sta()

loading data ... 

get data ... 



In [22]:
print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

# with h5py.File('../train_data_weight.h5', 'r') as f:
#         weight = np.array(f.get('weight'))
# train_w, test_w= train_test_split(weight, test_size=0.2, random_state=2020)

# train_w = np.squeeze(train_w)
# t_w = 10/np.log(train_w)


model.fit({'main_input': train_x },#,'aux_input': train_x_sta},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
#           class_weight = {0:0.698, 1:0.520, 2:0.671, 3:0.698, 4:0.945,5:0.890, 6:0.986, 7:1. , 8:0.808,9:0.657},
#           class_weight = {0:1.,1:2.159}
#           sample_weight = t_w,
          validation_data=({'main_input': test_x},#,'aux_input': test_x_sta},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])
# loss: 1.4593 - acc: 0.4002 - val_loss: 1.4072 - val_acc: 0.4215

lstm model fit...

Train on 720000 samples, validate on 180000 samples
Epoch 1/100
720000/720000 [==============================] - 2231s 3ms/step - loss: 1.4444 - acc: 0.4057 - val_loss: 1.3940 - val_acc: 0.4250
Epoch 2/100
  8704/720000 [..............................] - ETA: 34:30 - loss: 1.2387 - acc: 0.4994

KeyboardInterrupt: 

In [ ]:

# pre = model.predict(test_x,verbose=1)

# #评估结果
# from sklearn.metrics import confusion_matrix, classification_report
# y_ = np.reshape(np.argmax(test_y,axis=1),[-1])
# pre_ = np.reshape(np.argmax(pre, axis=1),[-1])
# #每个类的各项指标
# cm = confusion_matrix(y_, pre_)
# # np.set_printoptions(precision=3)
# cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# print(cm_normalized)
# print(classification_report(y_, pre_))